In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [2]:
import cv2
import numpy as np
import glob
from skimage import io

width=200
height=200


Data=[]
Target=[]

# benign cancer
file_list = glob.glob(r'/content/drive/My Drive/DataSet/BUSI/benign/*.png')

for i in file_list:
  im=cv2.imread(i)
  im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
  im=cv2.resize(im,(width,height))
  Data.append(im)
  Target.append(0)

file_list = glob.glob(r'/content/drive/My Drive/DataSet/BUSI/malignant/*.png')
# malignant cancer
for i in file_list:
  im=cv2.imread(i)
  im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
  im=cv2.resize(im,(width,height))
  Data.append(im)
  Target.append(1)

file_list = glob.glob(r'/content/drive/My Drive/DataSet/BUSI/normal/*.png')
# malignant cancer
for i in file_list:
  im=cv2.imread(i)
  im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
  im=cv2.resize(im,(width,height))
  Data.append(im)
  Target.append(2)


In [3]:
from tensorflow.keras.utils import to_categorical
Final_Target = to_categorical(Target)

In [ ]:
Final_Data=np.reshape(Data,(len(Data),height,width,1))

In [ ]:
# CNN LSTM
from keras import layers
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv2D,SeparableConv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import TimeDistributed
from keras.layers import LSTM
from keras.models import Model

myInput=layers.Input(shape=(height,width,1))
conv1=SeparableConv2D(64,3, activation='relu',padding='same')(myInput)
max1=MaxPooling2D(2)(conv1)
Norm1=layers.BatchNormalization()(max1)
drop1=layers.Dropout(0.2)(Norm1)
conv2=Conv2D(128,3, activation='relu',padding='same')(drop1)
max2=MaxPooling2D(2)(conv2)
drop2=layers.Dropout(0.2)(max2)
Time1=layers.TimeDistributed(Flatten())(drop2)
lstm1=LSTM(150)(Time1)
des1 =Dense(50, activation='relu')(lstm1)
Extract_Model = Model(myInput, des1)
out_layer =Dense(3, activation='softmax')(des1)
model = Model(myInput, out_layer)
model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

verbose, epochs, batch_size = 1, 10, 32
model.fit(Final_Data,Final_Target, epochs=epochs, batch_size=batch_size, verbose=verbose)

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 200, 200, 1)]     0         
                                                                 
 separable_conv2d_2 (Separab  (None, 200, 200, 64)     137       
 leConv2D)                                                       
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 100, 100, 64)     0         
 2D)                                                             
                                                                 
 batch_normalization_2 (Batc  (None, 100, 100, 64)     256       
 hNormalization)                                                 
                                                                 
 dropout_1 (Dropout)         (None, 100, 100, 64)      0         
                                                           

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Data, Target, test_size=0.3, random_state=42)

In [ ]:
from tensorflow.keras.utils import to_categorical
trainy = to_categorical(y_train)
testy = to_categorical(y_test)

In [ ]:
trainX=np.reshape(X_train,(len(X_train),height,width,1))
testX=np.reshape(X_test,(len(X_test),height,width,1))

In [ ]:
trainFeature=(Extract_Model.predict(trainX))
testFeature=(Extract_Model.predict(testX))

8/8 [==============================] - 9s 1s/step


In [ ]:
from sklearn import svm
classifier_svm = svm.LinearSVC()
classifier_svm.fit(trainFeature, y_train)

LinearSVC()

In [ ]:
from sklearn import tree
classifier_tree = tree.DecisionTreeClassifier()
classifier_tree.fit(trainFeature, y_train)

DecisionTreeClassifier()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier_Knn = KNeighborsClassifier(n_neighbors=3)
classifier_Knn.fit(trainFeature, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
from sklearn.ensemble import  VotingClassifier
classifier_Voting = VotingClassifier(estimators=[('SVM', classifier_svm), ('TREE', classifier_tree), ('KNN', classifier_Knn)],weights=[1,1,1])
classifier_Voting.fit(trainFeature, y_train)
Class_Voting=classifier_Voting.predict(testFeature)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
print(classification_report(y_test,Class_Voting))

              precision    recall  f1-score   support

           0       0.62      0.84      0.72       138
           1       0.44      0.23      0.30        60
           2       0.31      0.14      0.19        36

    accuracy                           0.58       234
   macro avg       0.46      0.40      0.40       234
weighted avg       0.53      0.58      0.53       234



In [ ]:
print('Accuracy= ' + str(round(accuracy_score(y_test, Class_Voting)*100)))


Accuracy= 58
